In [ ]:
import pandas as pd
from nltk.corpus import stopwords
import json


In [52]:
annotations_data = pd.read_csv(r'/Users/vcroopana/Downloads/summer2020/oscar_ads/ip/ad_annotations_oscars_national_edited.csv') 
annotations_data = annotations_data[['Advertiser', 'location', 'Ad', 'Keywords', 'Twitter_handle']]
annotations_data['Keywords'] = annotations_data['Ad'].str.cat(annotations_data['Keywords'], sep=" ")\
                                .str.cat(annotations_data['Twitter_handle'], sep=" ")\
                                .str.cat(annotations_data['Advertiser'], sep=" ")# concat the Ad, twitter handle and keywords fields 

annotations_data = annotations_data.fillna("") #location column has NaN values
annotations_data['advtsr_loc'] = annotations_data['Advertiser'].str.cat(annotations_data['location'], sep="_")

twitter_handles = annotations_data['Twitter_handle'].values.tolist()
annotations_data.head(2)


,Advertiser,location,Ad,Keywords,Twitter_handle,advtsr_loc
0,Cadillac_Escalade,(00:02:44),Escalade,"Escalade Cadillac, driving, red carpet, escala...",cadillac,Cadillac_Escalade_(00:02:44)
1,Kohl's,,Sale,"Sale kohl's, home sale, kohl's cash, kohls.com...",kohls,Kohl's_


In [53]:
stop = stopwords.words('english')
print(len(stop))
stop.extend(["best", "award", "love", "movie", "picture", "words", "actor", "actress", "red", "carpet", "oscar", "oscars"])
print(len(stop))

179
191


In [161]:

json_tweets = []
json_tweets_filtered = []
filePath = "/Users/vcroopana/Downloads/summer2020/oscar_data/ip/200209191649_academy_awards_20200210_vol"

for i in range(1,8):
# for i in range(1,3):

    filePathi = filePath+"-"+str(i)+".json"
    counter = 0
    with open(filePathi) as f:
        for line in f:
            #json.loads reads file content as string
            json_content = json.loads(line)
            #Add user id and user name fields to the json unlike in the nested 'user' object
            user_id = json_content['user']['id']        
            user_name = json_content['user']['name']
            user_descr = json_content['user']['description']
            tweet_id_str = json_content['id_str']
            created_at = json_content['created_at']
            text = json_content['text']
            if 'retweeted_status' in json_content:
                retweeted_status = json_content['retweeted_status']
            else:
                retweeted_status = 'NA'
            
            tweet_dict = {'tweet_id_str': tweet_id_str,'created_at':created_at, 'text':text, 
                          'retweeted_status':retweeted_status,
                          'user_id': user_id, 'user_name': user_name, 'user_descr': user_descr}            
            json_tweets_filtered.append(tweet_dict)
            counter = counter+1               
    print(str(counter)+" Tweets extracted from input json:"+ str(i))
    
print("Extracted tweets from all the jsons")

df_tweets_raw = pd.DataFrame(json_tweets_filtered)

100000 Tweets extracted from input json:1
100000 Tweets extracted from input json:2
100000 Tweets extracted from input json:3
100000 Tweets extracted from input json:4
100000 Tweets extracted from input json:5
100000 Tweets extracted from input json:6
37999 Tweets extracted from input json:7
Extracted tweets from all the jsons


In [162]:
#df_tweets = df_tweets_raw[['tweet_id_str', 'created_at', 'text', 'retweeted_status', 'user_id', 'user_name']]
df_tweets = df_tweets_raw[['tweet_id_str', 'created_at', 'text','user_id', 'user_name']]
df_tweets.head(2)


,tweet_id_str,created_at,text,user_id,user_name
0,1226677826239242242,Mon Feb 10 01:22:39 +0000 2020,RT @BlazeTV: Join the 31K+ people currently wa...,339831428,SeldenGADawgs
1,1226677826121879552,Mon Feb 10 01:22:38 +0000 2020,"RT @SportsTaxMan: $113,175: Amount celebs will...",3183656815,Kevin puff


In [ ]:
#clean tweets
import re
def removeMentions(text):

    textBeforeMention = text.partition("@")[0]
    textAfterMention = text.partition("@")[2]
    textAfterMention =  re.sub(r':', '', textAfterMention) #cadillac join the 31k
    tHandle = textAfterMention.partition(" ")[0].lower() #cadillac
#     if tHandle not in twitter_handles:
#         text = re.sub(r'@\w+', '', text) 
#     else:
#         text = textBeforeMention+ " " + textAfterMention     
    text = textBeforeMention+ " " + textAfterMention  
    return text
    
#TODO issue should've getting converted to shouldve couldn' t -> couldn
def cleanTweet(strinp):
    strinp = re.sub(r'RT', "", strinp) # Remove RT
    strinp = strinp.lower()
    
    stop_removed_list = [word for word in strinp.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_list])    
    text = re.sub('https?://[A-Za-z0-9./]+', ' ', stop_removed) # Remove URLs
    text = removeMentions(text)
    text = re.sub('[^\x00-\x7F]+', '', text) # Remove non-ASCII chars.
    text = re.sub('[^a-zA-Z]', ' ', text) # remove all other than alphabet chars 
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters     
    stop_removed_l = [word for word in text.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_l]) 
    return stop_removed

print(cleanTweet("RT @cadillacabc: Joinrt the 31K") )

In [ ]:
df_tweets['text_clean'] = df_tweets['text'].apply(lambda x: cleanTweet(x))
df_tweets.head(2)

In [85]:
from gensim import corpora

words = [tweet.split() for tweet in df_tweets['text_clean']]
print(len(words))
dictionary = corpora.Dictionary(words)
#words or df_tweets['text_clean'].list()
corpus = [dictionary.doc2bow(text) for text in words]

200000


LDA Model

In [61]:
def printTopics(model, nTopics, nWords):
    for i in range(0,nTopics):
        print(str(i)+":"+ str(model.show_topic(i, nWords)))
        #print(model.get_topic_terms(21,10))

In [ ]:
from gensim import models

nTopics = 50
model = models.ldamodel.LdaModel(corpus, num_topics= nTopics, id2word=dictionary, chunksize= 500, 
                                 alpha = 'auto', passes=30, minimum_probability =0.0001)

topics = model.print_topics(nTopics, num_words=15)
print(len(topics))

# for topic in topics:
#     print(topic)

In [ ]:
#####Results#####
#commercial - 21, google - 5
# google - 14
# taxes , pay - 15
# regina, gown cadillac - 62
# commercial, postmates, food - 44

#ntopics - 50, asyymmetric, chunksize= 500, alpha = 'auto', passes=30, minimum_probability =0.0001)

#samsung, phone - 13
#commercial - 
# cadillac - 38
# verizon, ad - 36


LDA MultiCore Model

In [ ]:
from gensim import models

nTopics = 150
nWords = 40
lda_model = models.ldamulticore.LdaMulticore(
                           corpus=corpus,
                           num_topics= nTopics,
                           id2word=dictionary,
                           chunksize=500,# chunksize=300,
                           workers=7, # Num. Processing Cores - 1
                           passes=20, #passes= 50
                           eval_every = 1,
                           per_word_topics=True)
lda_model.save('lda_model.model')

lda_topics = lda_model.print_topics(nTopics, num_words=15)
print(len(lda_topics))
printTopics(lda_model, nTopics, nWords)

HDP Model to identify the number of topics

In [ ]:
hdp_model = models.HdpModel(corpus, id2word=dictionary) #gave 150 topics
hdp_topics = hdp_model.print_topics(500)
print(len(hdp_topics))

In [ ]:
printTopics(hdp_model, 150, nWords=40)

TF-IDF corpus and Latent Semantic Indexing Model- tfidf since the words of interest are less frequent
LDA in general gives priority to high frequent words

In [ ]:
tfidf = models.TfidfModel(corpus) 
corpus_tfidf = tfidf[corpus]
print(len(corpus_tfidf))
# for doc in corpus_tfidf:
#     print(doc)
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=100)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]
# for doc in corpus_lsi:
#     print(doc)
printTopics(lsi_model, 50, nWords=40)


In [ ]:
lsi_topics = lsi_model.print_topics(100, 200)
i =0
for t in lsi_topics:
    print(t)
#     print(str(i)+","+ str(t))
    i = i+1

In [ ]:
nTopics = 50
lda_tf_idf = models.ldamulticore.LdaMulticore(
                           corpus=corpus_tfidf,
                           num_topics= nTopics,
                           id2word=dictionary,
                           chunksize=500,# chunksize=300,
                           workers=7, # Num. Processing Cores - 1
                           passes=20, #passes= 50
                           eval_every = 1,
                           per_word_topics=True)
lda_tf_idf.save('lda_tf_idf.model')

lda_tf_idf_topics = lda_tf_idf.print_topics(nTopics, num_words=50)
print(len(lda_tf_idf_topics))
printTopics(lda_tf_idf, nTopics, nWords = 40)

In [ ]:
lda_tf_idf_topics = lda_tf_idf.print_topics(nTopics, num_words=50)
print(len(lda_tf_idf_topics))
printTopics(lda_tf_idf, nTopics, nWords = 40)

In [ ]:
#LDA for ads

In [142]:
annotations_data['keywords_clean'] = annotations_data['Keywords'].apply(lambda x: cleanTweet(x))
# print(annotations_data.head(2))
ad_words = [keywrds.split() for keywrds in annotations_data['keywords_clean']]
print(len(ad_words))

tweet_words = [tweet.split() for tweet in df_tweets['text_clean']]
ad_words.extend(tweet_words)
print(len(ad_words))

ad_dictionary = corpora.Dictionary(ad_words)
ad_corpus = [ad_dictionary.doc2bow(text) for text in [keywrds.split() for keywrds in annotations_data['keywords_clean']]]


74
200074


In [ ]:
nTopics = 67
lda_ads_model = models.ldamulticore.LdaMulticore(
                           corpus=ad_corpus,
                           num_topics= nTopics,
                           id2word=ad_dictionary,
                           chunksize=2,# chunksize=300,
                           workers=7, # Num. Processing Cores - 1
                           passes=20, #passes= 50
                           eval_every = 1,
                           per_word_topics=True, minimum_probability =0.0001)

#results
#nyt, phone, flip, 

In [168]:
printTopics(lda_ads_model, nTopics, nWords = 10)

0:[('gbm', 2.4013067e-05), ('helpisnicee', 2.4013067e-05), ('ozforbernie', 2.4013067e-05), ('yuge', 2.4013067e-05), ('grandcalifornia', 2.4013067e-05), ('thesolid', 2.4013067e-05), ('kfilm', 2.4013067e-05), ('whatthekpop', 2.4013067e-05), ('epilepsy', 2.4013067e-05), ('finaldraftinc', 2.4013067e-05)]
1:[('peroni', 0.00344199), ('italia', 0.0025873808), ('peroniusa', 0.00087799784), ('beer', 0.00087799784), ('birra', 0.00087799784), ('motorcycle', 0.00087799784), ('beautifully', 0.00087799784), ('seyfried', 2.3766854e-05), ('johns', 2.3766854e-05), ('shrink', 2.3766854e-05)]
2:[('cadillac', 0.02044938), ('escalade', 0.012881591), ('com', 0.009721753), ('make', 0.008661226), ('makeyourway', 0.007275829), ('way', 0.0067076315), ('kohl', 0.0058151553), ('burgers', 0.003928001), ('king', 0.0037817524), ('regina', 0.0037817524)]
3:[('gbm', 2.4013067e-05), ('helpisnicee', 2.4013067e-05), ('ozforbernie', 2.4013067e-05), ('yuge', 2.4013067e-05), ('grandcalifornia', 2.4013067e-05), ('thesolid', 

46:[('gbm', 2.4013067e-05), ('helpisnicee', 2.4013067e-05), ('ozforbernie', 2.4013067e-05), ('yuge', 2.4013067e-05), ('grandcalifornia', 2.4013067e-05), ('thesolid', 2.4013067e-05), ('kfilm', 2.4013067e-05), ('whatthekpop', 2.4013067e-05), ('epilepsy', 2.4013067e-05), ('finaldraftinc', 2.4013067e-05)]
47:[('gbm', 2.4013067e-05), ('helpisnicee', 2.4013067e-05), ('ozforbernie', 2.4013067e-05), ('yuge', 2.4013067e-05), ('grandcalifornia', 2.4013067e-05), ('thesolid', 2.4013067e-05), ('kfilm', 2.4013067e-05), ('whatthekpop', 2.4013067e-05), ('epilepsy', 2.4013067e-05), ('finaldraftinc', 2.4013067e-05)]
48:[('gbm', 2.4013067e-05), ('helpisnicee', 2.4013067e-05), ('ozforbernie', 2.4013067e-05), ('yuge', 2.4013067e-05), ('grandcalifornia', 2.4013067e-05), ('thesolid', 2.4013067e-05), ('kfilm', 2.4013067e-05), ('whatthekpop', 2.4013067e-05), ('epilepsy', 2.4013067e-05), ('finaldraftinc', 2.4013067e-05)]
49:[('gbm', 2.4013067e-05), ('helpisnicee', 2.4013067e-05), ('ozforbernie', 2.4013067e-05),

In [130]:
tweet_words = [tweet.split() for tweet in df_tweets['text_clean']]
tweet_corpus = [ad_dictionary.doc2bow(text) for text in tweet_words]


In [166]:
def sort_tuple(tupList):  
    tupList.sort(key = lambda x: x[1])  
    return tupList  

i = 0
not_none_cat =0
for i in range(0, len(tweet_corpus)):    
    result = lda_ads_model.get_document_topics(tweet_corpus[i])
    sort_tuple(result)
    topic = result[-1]    
#     print(str(topic[0])+ "-" + str(topic[1])+" - "+ str(tweet_words[i]))
    if topic[1] >= 0.7:     
        print(str(topic[0])+ "-" + str(topic[1])+" - "+ str(tweet_words[i]))
        not_none_cat = not_none_cat + 1
    
    if i % 5000 ==0 :
        print(str(i)+" th iteration...")
        print("#" + str(not_none_cat)+ " not nones found")    
    
    i = i+1
    
print(not_none_cat)

0 th iteration...
#0 not nones found
54-0.7536976 - ['horkv', 'watch', 'online', 'amp', 'free', 'watch', 'live', 'stream', 'show', 'free', 'streaming', 'free', 'link', 'gt']
9-0.7537259 - ['dude', 'samsung', 'unveiled', 'upcoming', 'foldable', 'phone', 'unpacked']
9-0.7537204 - ['samsung', 'casually', 'showed', 'galaxy', 'flip']
18-0.8358089 - ['postmates', 'used', 'robs', 'make', 'fast', 'make', 'slow', 'commercial', 'inner', 'crate', 'digger', 'jumped', 'ft', 'fr']
9-0.7537234 - ['know', 'would', 'trust', 'motorolla', 'flip', 'phone', 'trust', 'samsung']
36-0.7537254 - ['james', 'cameron', 'appearing', 'rolex', 'commercial', 'working', 'avatar', 'sequels']
17-0.75369775 - ['yes', 'yes', 'yes', 'right', 'ad']
39-0.8029806 - ['regina', 'king', 'get', 'another', 'regina', 'king', 'class', 'class', 'class']
57-0.7537289 - ['notryanlynch', 'hulu', 'nope', 'bailed', 'hulu', 'went', 'youtubetv', 'hulu', 'kept', 'screwing', 'terrible', 'sup']
56-0.75372344 - ['rashadrobinson', 'presented', '

24-0.7537047 - ['voguemagazine', 'natalie', 'portman', 'wears', 'dior', 'haute', 'couture', 'cape', 'embroidered', 'names', 'th']
54-0.7536973 - ['guys', 'join', 'us', 'right', 'free', 'hotmicinc', 'app', 'lets', 'watch', 'together']
24-0.75370294 - ['vulture', 'miss', 'natalie', 'portman', 'showed', 'show']
17-0.7536977 - ['yes', 'yes', 'yes']
9-0.80297774 - ['samsung', 'aired', 'galaxy', 'flip', 'commercial', 'even', 'announced', 'phone']
24-0.7537047 - ['voguemagazine', 'natalie', 'portman', 'wears', 'dior', 'haute', 'couture', 'cape', 'embroidered', 'names', 'th']
39-0.7537249 - ['njerieaton', 'someone', 'immediately', 'make', 'romantic', 'comedy', 'starring', 'regina', 'king', 'brad', 'pitt', 'post', 'haste']
9-0.80297774 - ['verge', 'samsung', 'aired', 'galaxy', 'flip', 'commercial', 'even', 'announced', 'phone']
25000 th iteration...
#85 not nones found
7-0.8358125 - ['give', 'margot', 'give', 'margot', 'give', 'margot', 'give', 'margot', 'give', 'margot']
24-0.75370467 - ['thef

37-0.7537068 - ['disney', 'theyre', 'movies', 'theyre', 'memories', 'stream', 'disneyplus', 'disneyplus']
54-0.75369763 - ['jessiemaltin', 'guys', 'join', 'us', 'right', 'free', 'hotmicinc', 'app', 'lets', 'watch', 'together', 'academyawards', 'os']
37-0.7537069 - ['disney', 'theyre', 'movies', 'theyre', 'memories', 'stream', 'disneyplus', 'disneyplus']
45-0.75370526 - ['arikhanson', 'liked', 'delta', 'americanexpress', 'ad', 'coolmomstatus', 'im', 'gold', 'amex', 'holder', 'makes', 'wa']
24-0.7537047 - ['thefashioncourt', 'natalie', 'portman', 'wore', 'dior', 'spring', 'haute', 'couture', 'gown', 'full', 'length', 'cape', 'embroidered', 'names']
37-0.7537164 - ['idinamenzel', 'elsas', 'slayed', 'frozen', 'disneyfrozen', 'theacademy', 'disney', 'disney']
50000 th iteration...
#154 not nones found
24-0.7537047 - ['thefashioncourt', 'natalie', 'portman', 'wore', 'dior', 'spring', 'haute', 'couture', 'gown', 'full', 'length', 'cape', 'embroidered', 'names']
36-0.7796032 - ['rolex', 'great

24-0.7537047 - ['mikebeauvais', 'natalie', 'portman', 'wearing', 'christian', 'dior', 'timothee', 'chalamet', 'wearing', 'paulie', 'walnuts']
37-0.7537069 - ['disney', 'theyre', 'movies', 'theyre', 'memories', 'stream', 'disneyplus', 'disneyplus']
37-0.75370705 - ['brunaneptune', 'bru', 'bru', 'spoils', 'us', 'giving', 'us', 'whole', 'ass', 'disney', 'hooligans', 'spoiled', 'spoiled', 'spoiled']
37-0.7537069 - ['disney', 'theyre', 'movies', 'theyre', 'memories', 'stream', 'disneyplus', 'disneyplus']
17-0.8029588 - ['yes', 'yes', 'yes', 'yes', 'yesyesyeysyesy']
17-0.7536977 - ['yes', 'yes', 'yes']
24-0.7537047 - ['thefashioncourt', 'natalie', 'portman', 'wore', 'dior', 'spring', 'haute', 'couture', 'gown', 'full', 'length', 'cape', 'embroidered', 'names']
17-0.75369775 - ['yes', 'yes', 'yes', 'queen']
37-0.7537069 - ['disney', 'theyre', 'movies', 'theyre', 'memories', 'stream', 'disneyplus', 'disneyplus']
17-0.75369775 - ['yes', 'yes', 'yes']
17-0.7536977 - ['yes', 'yes', 'yes']
54-0.75

95000 th iteration...
#307 not nones found
24-0.7537047 - ['voguemagazine', 'natalie', 'portman', 'wears', 'dior', 'haute', 'couture', 'cape', 'embroidered', 'names', 'th']
24-0.7536954 - ['loventhunders', 'taika', 'waititi', 'giving', 'mjolnir', 'natalie', 'portman', 'natalie', 'giving', 'taika', 'thing', 'matters']
24-0.75369537 - ['loventhunders', 'taika', 'waititi', 'giving', 'mjolnir', 'natalie', 'portman', 'natalie', 'giving', 'taika', 'thing', 'matters']
51-0.7537216 - ['hilarious', 'seeing', 'runjewels', 'nobody', 'speak', 'lead', 'cadillac', 'commercial', 'know', 'first', 'kicks']
17-0.7536977 - ['yes', 'yes', 'yes']
24-0.7536954 - ['loventhunders', 'taika', 'waititi', 'giving', 'mjolnir', 'natalie', 'portman', 'natalie', 'giving', 'taika', 'thing', 'matters']
24-0.75369537 - ['loventhunders', 'taika', 'waititi', 'giving', 'mjolnir', 'natalie', 'portman', 'natalie', 'giving', 'taika', 'thing', 'matters']
24-0.7537047 - ['voguemagazine', 'natalie', 'portman', 'wears', 'dior', '

...

24-0.7536954 - ['loventhunders', 'taika', 'waititi', 'giving', 'mjolnir', 'natalie', 'portman', 'natalie', 'giving', 'taika', 'thing', 'matters']
24-0.75369537 - ['loventhunders', 'taika', 'waititi', 'giving', 'mjolnir', 'natalie', 'portman', 'natalie', 'giving', 'taika', 'thing', 'matters']
37-0.75371647 - ['three', 'favorite', 'disney', 'movies', 'disclaimer', 'racism', 'disney', 'im', 'crying', 'right', 'conflicted']
24-0.75370467 - ['voguemagazine', 'natalie', 'portman', 'wears', 'dior', 'haute', 'couture', 'cape', 'embroidered', 'names', 'th']
24-0.7536954 - ['davechensky', 'really', 'nice', 'natalie', 'portman', 'invite', 'valet', 'driver', 'stage', 'introduce', 'adapted', 'screenplay']
17-0.75369775 - ['holy', 'shit', 'learningtoskateboardinawarzone', 'yes', 'yes', 'yes']
24-0.7536954 - ['davechensky', 'really', 'nice', 'natalie', 'portman', 'invite', 'valet', 'driver', 'stage', 'introduce', 'adapted', 'screenplay']
37-0.75370693 - ['disney', 'theyre', 'movies', 'theyre', 'memor

9-0.7537204 - ['samsung', 'teased', 'galaxy', 'flip', 'commercial', 'break']
24-0.75370467 - ['voguemagazine', 'natalie', 'portman', 'wears', 'dior', 'haute', 'couture', 'cape', 'embroidered', 'names', 'th']
9-0.80297774 - ['samsung', 'aired', 'galaxy', 'flip', 'commercial', 'even', 'announced', 'phone']
24-0.8029569 - ['itsjustanx', 'taika', 'waititi', 'announcing', 'natalie', 'portman', 'mighty', 'thor', 'natalie', 'portman', 'announcing', 'taika', 'waititi', 'winner', 'fu']
24-0.802957 - ['itsjustanx', 'taika', 'waititi', 'announcing', 'natalie', 'portman', 'mighty', 'thor', 'natalie', 'portman', 'announcing', 'taika', 'waititi', 'winner', 'fu']
24-0.8029569 - ['itsjustanx', 'taika', 'waititi', 'announcing', 'natalie', 'portman', 'mighty', 'thor', 'natalie', 'portman', 'announcing', 'taika', 'waititi', 'winner', 'fu']
24-0.753703 - ['vulture', 'miss', 'natalie', 'portman', 'showed', 'show']
24-0.75369537 - ['loventhunders', 'taika', 'waititi', 'giving', 'mjolnir', 'natalie', 'portma

24-0.75369537 - ['loventhunders', 'taika', 'waititi', 'giving', 'mjolnir', 'natalie', 'portman', 'natalie', 'giving', 'taika', 'thing', 'matters']
39-0.75372684 - ['okay', 'regina', 'king', 'buy', 'escalade', 'asap']
24-0.7536954 - ['loventhunders', 'taika', 'waititi', 'giving', 'mjolnir', 'natalie', 'portman', 'natalie', 'giving', 'taika', 'thing', 'matters']
24-0.7537047 - ['voguemagazine', 'natalie', 'portman', 'wears', 'dior', 'haute', 'couture', 'cape', 'embroidered', 'names', 'th']
39-0.8358172 - ['king', 'king', 'king', 'king', 'king']
24-0.8029569 - ['itsjustanx', 'taika', 'waititi', 'announcing', 'natalie', 'portman', 'mighty', 'thor', 'natalie', 'portman', 'announcing', 'taika', 'waititi', 'winner', 'fu']
9-0.7537204 - ['samsung', 'teased', 'galaxy', 'flip', 'amp', 'commercial', 'break', 'androidcentral']
24-0.7536954 - ['loventhunders', 'taika', 'waititi', 'giving', 'mjolnir', 'natalie', 'portman', 'natalie', 'giving', 'taika', 'thing', 'matters']
40-0.80298156 - ['really', 

185000 th iteration...
#749 not nones found
24-0.802957 - ['itsjustanx', 'taika', 'waititi', 'announcing', 'natalie', 'portman', 'mighty', 'thor', 'natalie', 'portman', 'announcing', 'taika', 'waititi', 'winner', 'fu']
9-0.80297774 - ['samsung', 'galaxy', 'flip', 'phone', 'officially', 'revealed', 'commercial']
24-0.7537047 - ['voguemagazine', 'natalie', 'portman', 'wears', 'dior', 'haute', 'couture', 'cape', 'embroidered', 'names', 'th']
24-0.7536954 - ['loventhunders', 'taika', 'waititi', 'giving', 'mjolnir', 'natalie', 'portman', 'natalie', 'giving', 'taika', 'thing', 'matters']
39-0.7537232 - ['regina', 'king', 'better', 'swap', 'caddy', 'ev', 'shes', 'actually', 'much', 'driving']
24-0.7536956 - ['syfyfangrrls', 'pretending', 'lauradern', 'win', 'partially', 'jurassic', 'park', 'tbh']
24-0.8029569 - ['itsjustanx', 'taika', 'waititi', 'announcing', 'natalie', 'portman', 'mighty', 'thor', 'natalie', 'portman', 'announcing', 'taika', 'waititi', 'winner', 'fu']
24-0.7537047 - ['voguem

24-0.75369537 - ['loventhunders', 'taika', 'waititi', 'giving', 'mjolnir', 'natalie', 'portman', 'natalie', 'giving', 'taika', 'thing', 'matters']
39-0.80297977 - ['indoob', 'imagine', 'regina', 'king', 'driving', 'show', 'cadillac']
24-0.7536954 - ['syfyfangrrls', 'pretending', 'lauradern', 'win', 'partially', 'jurassic', 'park', 'tbh']
195000 th iteration...
#817 not nones found
24-0.80295694 - ['itsjustanx', 'taika', 'waititi', 'announcing', 'natalie', 'portman', 'mighty', 'thor', 'natalie', 'portman', 'announcing', 'taika', 'waititi', 'winner', 'fu']
24-0.7536954 - ['syfyfangrrls', 'pretending', 'lauradern', 'win', 'partially', 'jurassic', 'park', 'tbh']
24-0.7536955 - ['historydramas', 'anne', 'boleyn', 'portrayed', 'natalie', 'portman', 'claire', 'foy', 'natalie', 'dormer']
37-0.7537259 - ['farringt', 'need', 'disney', 'princess', 'whos', 'whore', 'need', 'disney', 'princess', 'whos', 'whore', 'need', 'disney', 'princess', 'whos', 'whore']
24-0.8029569 - ['itsjustanx', 'taika', '

In [ ]:
#pip install pyLDAvis

In [160]:
import pyLDAvis.gensim

lda_ad_display = pyLDAvis.gensim.prepare(lda_ads_model, ad_corpus, ad_dictionary, sort_topics=True, mds='mmds')
pyLDAvis.display(lda_ad_display)


In [140]:
lda_display = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics=True,  mds='mmds')
pyLDAvis.display(lda_display)
